In [1]:
import logging as log
import re
from riboraptor.helpers import path_leaf
from riboraptor.cutadapt_to_json import cutadapt_to_json

/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
1123188 2711596 3697681 4992604 5950222 6435064 7716123 8404354 9348380 0430291

In [ ]:
regexes = {
    'bp_processed': "Total basepairs processed:\s*([\d,]+) bp",
    'bp_written': "Total written \(filtered\):\s*([\d,]+) bp",
    'quality_trimmed': "Quality-trimmed:\s*([\d,]+) bp",
    'r_processed': "Total reads processed:\s*([\d,]+)",
    'r_with_adapters': "Reads with adapters:\s*([\d,]+)"
}

In [ ]:

def fastqc_to_json(filepath):
    #fh = open('/staging/as/skchoudh/re-ribo-analysis/hg38/SRP010679_fixed_adapters/preprocessed_step1/SRR403882.fastq.gz_trimming_report.txt', 'r')
    fh = open('/staging/as/skchoudh/re-ribo-analysis/hg38/SRP010679_fixed_adapters/preprocessed/SRR403882_trimmed_trimmed.fq.gz_trimming_report.txt', 'r')


    cutadapt_version = '1.7'
    trim_info = {}
    length_counts = {}
    length_exp = {}
    length_obsexp = {}
    adapters = {}
    sample = None
    for l in fh:
        if 'cutadapt' in l:
            sample = None
            c_version = re.match(r'This is cutadapt ([\d\.]+)', l)   
        if l.startswith('Used user'):
            print(l)
        if l.startswith('Command line parameters'):
            sample = l.split()[-1]            
            sample = path_leaf(sample).replace('.fq.gz', '').replace('.fastq.gz', '')
            if sample in trim_info:
                log.debug("Duplicate sample name found! Overwriting: {}".format(sample))
            trim_info[sample] = dict()
        if sample is not None:
            for k, r in regexes.items():
                match = re.search(r, l)
                if match:
                    trim_info[sample][k] = int(match.group(1).replace(',', ''))

            if '===' in l:
                log_section = l.strip().strip('=').strip()
            if l.startswith('Sequence:'):
                plot_sname = '{} - {}'.format(sample, log_section)
                adapters[plot_sname] = l.split(';')[0].strip('Sequence: ')

            # Histogram showing lengths trimmed
            if 'length' in l and 'count' in l and 'expect' in l:
                plot_sname = sample
                if log_section is not None:
                    plot_sname = '{} - {}'.format(sample, log_section)
                length_counts[plot_sname] = dict()
                length_exp[plot_sname] = dict()
                length_obsexp[plot_sname] = dict()
                # Nested loop to read this section while the regex matches
                for l in fh:
                    r_seqs = re.search("^(\d+)\s+(\d+)\s+([\d\.]+)", l)
                    if r_seqs:
                        a_len = int(r_seqs.group(1))
                        length_counts[plot_sname][a_len] = int(r_seqs.group(2))
                        length_exp[plot_sname][a_len] = float(r_seqs.group(3))
                        if float(r_seqs.group(3)) > 0:
                            length_obsexp[plot_sname][a_len] = float(r_seqs.group(2)) / float(r_seqs.group(3))
                        else:                        
                            length_obsexp[plot_sname][a_len] = float(r_seqs.group(2))
                    else:
                        break

    json_data = {
        'adapters': adapters,
        'trim_info': trim_info,
        'length_exp': length_exp,
        'length_obsexp': length_obsexp,             
        'length_counts': length_counts,
    }
    


In [ ]:
fastqc_to_json('x')

In [2]:
cutadapt_to_json('/staging/as/skchoudh/re-ribo-analysis/hg38/SRP010679_fixed_adapters/preprocessed/SRR403882_trimmed_trimmed.fq.gz_trimming_report.txt')

{'adapters': 'User provided',
 'length_counts': {},
 'length_exp': {},
 'length_obsexp': {},
 'trim_info': {}}

In [ ]:
more /staging/as/skchoudh/re-ribo-analysis/hg38/SRP010679_fixed_adapters/preprocessed/SRR403882_trimmed_trimmed.fq.gz_trimming_report.txt